In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from utility import *

# Set_3

Challenge 17

In [ ]:
random_string_list = [b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93']

key = generate_random_aes_key()
IV = generate_random_aes_key()
block_size = 16

In [ ]:
# function 1
def encrypt_CBC_17() -> bytes:
    cipher_text = random.choice(random_string_list)
    cipher_text = padding_PKCS7(b'A' * 14, block_size)
    encrypted_text = encrypt_CBC(cipher_text, key, block_size, IV)
    return encrypted_text

# function 2
def check_padding_is_valid(encrypted_bytes: bytes) -> bool:
    try:
        decrypted_bytes = decrypt_CBC(encrypted_bytes, key, block_size, IV, False)
    except:
        return False
    target_bytes = decrypted_bytes[-1]
    #print(f'decrypted_bytes {decrypted_bytes}')
    #print(f'target_bytes {target_bytes}')
    if decrypted_bytes[-target_bytes:] == decrypted_bytes[-1:] * target_bytes:
        return True
    return False

In [ ]:
encrypted_bytes = encrypt_CBC_17()
print(encrypted_bytes)
check_padding_is_valid(encrypted_bytes)

b' D;s\xcb(\xadt\x95\x81<\xc0\x03\xc6\xff\x04V\x13\x8fT\xf3\xcf "gM\xa7\xc1r\xfd\x0e\xe5'
b'AAAAAAAAAAAAAA\x02\x02\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10' 16


True

In [ ]:
def r(c0, c1, c2, intermiedate_state, i):
    """
    Recursive function that performs a padding oracle attack.

    Args:
        c0 (bytes): The first part of the ciphertext.
        c1 (bytes): The second part of the ciphertext.
        c2 (bytes): The third part of the ciphertext.
        intermiedate_state (bytes): The intermediate state of the attack.
        i (int): The current iteration of the attack.

    Returns:
        bytes: The intermediate state of the attack if successful, None otherwise.
    """
    if len(intermiedate_state) == 16:
        return intermiedate_state
    suffix = xor_bytes(intermiedate_state, i.to_bytes(1, 'big') * (i - 1)) if i > 1 else b''
    print(suffix, i)
    for j in range(256):
        modified_encrypted_bytes = c0 + c1[:-i] + j.to_bytes(1, 'big') + suffix + c2
        # print(len(modified_encrypted_bytes))
        # print(modified_encrypted_bytes)
        if check_padding_is_valid(modified_encrypted_bytes):
            new_intermiedate_state = (j ^ i).to_bytes(1, 'big') + intermiedate_state
            next = r(c0, c1, c2, new_intermiedate_state, i + 1)
            if next:
                return next

In [ ]:
block_list = [encrypted_bytes[i:i + block_size] for i in range(0, len(encrypted_bytes), block_size)]
result = b''
for i in range(1, len(block_list)):
    c0 = b''.join(block_list[:-i-1])
    c1 = block_list[-i-1]
    c2 = block_list[-i]
    IS = r(c0, c1, c2, b'', 1)
    result = xor_bytes(IS, c1) + result

result = xor_bytes(r(b'', IV, c1, b'', 1), IV) + result
print(unpadding_PKCS7(result))

1
b'' 1
b'\x07' 2


TypeError: 'NoneType' object is not iterable

Challenge 18

In [ ]:
cipher_text = base64.b64decode('L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==')
key = b'YELLOW SUBMARINE'
nonce = 0
print(exec_CTR(cipher_text, key, nonce))

b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "


Challenge 19
Challenge 20

In [ ]:
b64_text_list = '''SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==
Q29taW5nIHdpdGggdml2aWQgZmFjZXM=
RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==
RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=
SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk
T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=
UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=
T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl
VG8gcGxlYXNlIGEgY29tcGFuaW9u
QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==
QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=
QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==
QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=
QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=
VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==
SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==
SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==
VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==
V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==
V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==
U2hlIHJvZGUgdG8gaGFycmllcnM/
VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=
QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=
VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=
V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=
SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==
U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==
U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=
VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==
QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu
SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=
VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs
WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=
SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0
SW4gdGhlIGNhc3VhbCBjb21lZHk7
SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=
VHJhbnNmb3JtZWQgdXR0ZXJseTo=
QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4='''.split('\n')
''' get data '''
bytes_list = []
for text in b64_text_list:
    bytes_list.append(base64.b64decode(text))

''' create encrypted data list with ctr mode '''
nonce = 0
key = generate_random_aes_key()
encrypted_btyes_list = []
for plain_bytes in bytes_list:
    encrypted_btyes_list.append(exec_CTR(plain_bytes, key, nonce))

In [ ]:
# printable_text = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ-?!"\',. :;'
# printable_text = "fHtcuivObUICnWSgpYr ?T,eyah';-A:EdwmkosPBF.l"
printable_chars = bytes(string.printable, 'ascii')
# printable_chars = bytes(printable_text, 'ascii')
all_possible_bytes_for_keystream = defaultdict(list)

''' loop each char from start '''
for i in range(38):
    ''' try all possible byte '''
    for possible_byte in range(256):
        unprintable_flag = False
        for encrypted_btyes in encrypted_btyes_list:
            if len(encrypted_btyes) <= i:
                continue
            if encrypted_btyes[i] ^ possible_byte not in printable_chars:
                unprintable_flag = True
                break
        if unprintable_flag:
            continue
        all_possible_bytes_for_keystream[i].append(possible_byte.to_bytes(1, 'little'))

In [ ]:
count = 1
for key, val in all_possible_bytes_for_keystream.items():
    print(key, len(val), val)
    count *= len(val)
print(count)

0 82 [b'\x80', b'\x81', b'\x82', b'\x83', b'\x84', b'\x85', b'\x86', b'\x87', b'\x88', b'\x89', b'\x8a', b'\x8b', b'\x8c', b'\x8d', b'\x8e', b'\x8f', b'\x90', b'\x91', b'\x92', b'\x93', b'\x94', b'\x95', b'\x96', b'\x97', b'\x98', b'\x99', b'\x9a', b'\x9b', b'\x9c', b'\x9d', b'\x9e', b'\x9f', b'\xc0', b'\xc1', b'\xc2', b'\xc3', b'\xc4', b'\xc6', b'\xc7', b'\xca', b'\xcd', b'\xce', b'\xd0', b'\xd1', b'\xd2', b'\xd6', b'\xd7', b'\xd8', b'\xdb', b'\xdc', b'\xe0', b'\xe1', b'\xe2', b'\xe3', b'\xe4', b'\xe5', b'\xe6', b'\xe7', b'\xe8', b'\xe9', b'\xea', b'\xeb', b'\xec', b'\xed', b'\xee', b'\xef', b'\xf0', b'\xf1', b'\xf2', b'\xf3', b'\xf4', b'\xf5', b'\xf6', b'\xf7', b'\xf8', b'\xf9', b'\xfa', b'\xfb', b'\xfc', b'\xfd', b'\xfe', b'\xff']
1 58 [b'\x00', b'\x01', b'\x02', b'\x03', b'\x04', b'\x05', b'\x06', b'\x07', b'\t', b'\n', b'\x0b', b'\x0c', b'\r', b'\x0e', b'\x10', b'\x16', b'\x17', b'\x19', b'\x1a', b'\x1d', b'\x1e', b'\x1f', b'(', b')', b'+', b'.', b'/', b'@', b'A', b'B', b'C', b'D'

In [ ]:
import challenge3 as c3
score_for_each_possible_bytes = []

for i, val in all_possible_bytes_for_keystream.items():
    score_list = []
    for possible_byte in val:
        score_input_text = ''
        for encrypted_btyes in encrypted_btyes_list:
            if len(encrypted_btyes) <= i:
                continue
            score_input_text += chr(encrypted_btyes[i] ^ possible_byte[0])
        score = c3.get_score(score_input_text)
        score_list.append((possible_byte, score))
    score_list.sort(key=lambda x: x[1], reverse=True)
    score_for_each_possible_bytes.append((i, score_list))

In [ ]:
for x in score_for_each_possible_bytes:
    print(x)

(0, [(b'\xc3', 226.11), (b'\xe3', 226.11), (b'\xc4', 199.75), (b'\xe4', 199.75), (b'\xe5', 195.56), (b'\xc7', 188.0), (b'\xe7', 188.0), (b'\xd8', 171.02), (b'\xf8', 171.02), (b'\xf9', 164.21), (b'\xd2', 164.19), (b'\xf2', 164.19), (b'\xce', 159.54), (b'\xee', 159.54), (b'\xfe', 151.7), (b'\xd6', 149.75), (b'\xf6', 149.75), (b'\xc2', 145.27), (b'\xe2', 145.27), (b'\xff', 138.51), (b'\xd1', 133.6), (b'\xf1', 133.6), (b'\xef', 131.87), (b'\xf5', 127.79), (b'\x82', 126), (b'\x97', 126), (b'\xcd', 121.16), (b'\xed', 121.16), (b'\xc0', 113.64), (b'\xe0', 113.64), (b'\xdb', 111.24), (b'\xfb', 111.24), (b'\xe9', 107.23), (b'\xc6', 106.53), (b'\xe6', 106.53), (b'\xeb', 105.84), (b'\xf3', 98.9), (b'\xd0', 96.33), (b'\xf0', 96.33), (b'\x8b', 90), (b'\xca', 89.74), (b'\xea', 89.74), (b'\xc1', 87.99), (b'\xe1', 87.99), (b'\xec', 87.54), (b'\xf4', 83.04), (b'\xe8', 78.71), (b'\xfa', 73.66), (b'\xd7', 72.42), (b'\xf7', 72.42), (b'\x8a', 72), (b'\xdc', 64.39), (b'\xfc', 64.39), (b'\xfd', 54.37), (b'\x

In [ ]:
predicted_key_stream = b''
for x in score_for_each_possible_bytes:
    if len(predicted_key_stream) == 31:
        predicted_key_stream += x[1][1][0]
    elif len(predicted_key_stream) == 33:
        predicted_key_stream += x[1][5][0]
    elif len(predicted_key_stream) == 34:
        predicted_key_stream += x[1][3][0]
    elif len(predicted_key_stream) == 35:
        predicted_key_stream += x[1][8][0]
    elif len(predicted_key_stream) == 36:
        predicted_key_stream += x[1][13][0]
    elif len(predicted_key_stream) == 37:
        predicted_key_stream += x[1][54][0]
    else:
        predicted_key_stream += x[1][0][0]

for encrypted_btyes in encrypted_btyes_list:
    print(xor_bytes(encrypted_btyes, predicted_key_stream))

b'i have met them at close of day'
b'coming with vivid faces'
b'from counter or desk among grey'
b'eighteenth-century houses.'
b'i have passed with a nod of the hEuR'
b'or polite meaningless words,'
b'or have lingered awhile and said'
b'polite meaningless words,'
b'and thought before I had done'
b'of a mocking tale or a gibe'
b'to please a companion'
b'around the fire at the club,'
b'being certain that they and I'
b'but lived where motley is worn:'
b'all changed, changed utterly:'
b'a terrible beauty is born.'
b"that woman's days were spent"
b'in ignorant good will,'
b'her nights in argument'
b'until her voice grew shrill.'
b'what voice more sweet than hers'
b'when young and beautiful,'
b'she rode to harriers?'
b'this man had kept a school'
b'and rode our winged horse.'
b'this other his helper and friend'
b'was coming into his force;'
b'he might have won fame in the end\x0c'
b'so sensitive his nature seemed,'
b'so daring and sweet his thought.'
b'this other man I had dreamed'
b'a drunk

Challenge 22

In [ ]:
import time
import random
import challenge21 as c21

def routine():
    wait_time = random.randint(1,20)
    #time.sleep(wait_time)
    print(f"first_sleep_time: {wait_time}s")

    current_timestamp = int(time.time())
    MT19937 = c21.MT19937(current_timestamp)
    print(f"Seeds: {current_timestamp}")

    wait_time = random.randint(1,20)
    #time.sleep(wait_time)
    print(f"second_sleep_time: {wait_time}s")

    return MT19937.rand()

random_num = routine()

output_list = dict()
for i in range(1000):
    current_timestamp = int(time.time())
    seed = current_timestamp - i
    MT19937 = c21.MT19937(seed)
    output_list[MT19937.rand()] = seed

if random_num in output_list:
    print(f"found seed : {output_list[random_num]}")
else:
    print(f"seed not found.")


first_sleep_time: 5s
Seeds: 1689426803
second_sleep_time: 14s
found seed : 1689426803


Challenge 23

In [ ]:
import challenge21 as c21

def untemper(y):
    (u, d) = (11, 0xFFFFFFFF)
    (s, b) = (7, 0x9D2C5680)
    (t, c) = (15, 0xEFC60000)
    l = 18

    def int_to_bit_list(x):
        return [int(b) for b in '{:032b}'.format(x)]

    def bit_list_to_int(l):
        return int(''.join(str(x) for x in l), base=2)

    def invert_shift_mask_xor(y, direction, shift, mask=0xFFFFFFFF):
        y = int_to_bit_list(y)
        mask = int_to_bit_list(mask)

        if direction == 'left':
            y.reverse()
            mask.reverse()
        else:
            assert direction == 'right'

        x = [None]*32
        for n in range(32):
            if n < shift:
                x[n] = y[n]
            else:
                x[n] = y[n] ^ (mask[n] & x[n-shift])

        if direction == 'left':
            x.reverse()

        return bit_list_to_int(x)

    xx = y
    xx = invert_shift_mask_xor(xx, direction='right', shift=l)
    xx = invert_shift_mask_xor(xx, direction='left', shift=t, mask=c)
    xx = invert_shift_mask_xor(xx, direction='left', shift=s, mask=b)
    xx = invert_shift_mask_xor(xx, direction='right', shift=u, mask=d)

    return xx

In [ ]:
def p(a, t):
    #a = a & 0xFFFFFFFF
    print("{0:032b}".format(a) + "  " + t)

y1 = 0x01020304
p(y1, 'y1')
print('')

y2 = y1 ^ ((y1 >> 11) & 0xFFFFFFFF)
p(y1 >> 11, 'y1 >> 11')
p(0xFFFFFFFF, '0xFFFFFFFF')
p((y1 >> 11) & 0xFFFFFFFF, '(y1 >> 11) & 0xFFFFFFFF')
p(y2, 'y1 ^ ((y1 >> 11) & 0xFFFFFFFF)')
print('')

y3 = y2 ^ ((y2 << 7) & 0x9D2C5680)
p(y2 << 7, 'y2 << 7')
p(0x9D2C5680, '0x9D2C5680')
p((y2 << 7) & 0x9D2C5680, '(y2 << 7) & 0x9D2C5680')
p(y3, 'y2 ^ ((y2 << 7) & 0x9D2C5680)')
print('')

y4 = y3 ^ ((y3 << 15) & 0xEFC60000)
p(y3 << 15, 'y3 << 15')
p(0xEFC60000, '0xEFC60000')
p((y3 << 15) & 0xEFC60000, '(y3 << 15) & 0xEFC60000')
p(y4, 'y3 ^ ((y3 << 15) & 0xEFC60000)')
print('')

z = y4 ^ (y4 >> 18)
p(y4 >> 18, 'y4 >> 18')
p(z, 'y4 ^ (y4 >> 18)')
print(f'result: {z}')

00000001000000100000001100000100  y1

00000000000000000010000001000000  y1 >> 11
11111111111111111111111111111111  0xFFFFFFFF
00000000000000000010000001000000  (y1 >> 11) & 0xFFFFFFFF
00000001000000100010001101000100  y1 ^ ((y1 >> 11) & 0xFFFFFFFF)

10000001000100011010001000000000  y2 << 7
10011101001011000101011010000000  0x9D2C5680
10000001000000000000001000000000  (y2 << 7) & 0x9D2C5680
10000000000000100010000101000100  y2 ^ ((y2 << 7) & 0x9D2C5680)

10000000000000100010000101000100000000000000000  y3 << 15
11101111110001100000000000000000  0xEFC60000
00000000100000100000000000000000  (y3 << 15) & 0xEFC60000
10000000100000000010000101000100  y3 ^ ((y3 << 15) & 0xEFC60000)

00000000000000000010000000100000  y4 >> 18
10000000100000000000000101100100  y4 ^ (y4 >> 18)
result: 2155872612


In [ ]:
import challenge21 as c21

seed = 12345678
rand_num = c21.MT19937(12345678).rand()
print(rand_num)
print(untemper(rand_num))

399782711
12345678


Challenge 24

In [ ]:
import challenge24 as c24
c24.check_is_MT19937_product_with_current_time(c24.generate_password_reset_token(32))

seed found: 1689426804


True